### Prep ERA5 surface heat flux data

- **Data Source**: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview
- **Data Description**: European Centre for Medium-Range Weather Forecasts, monthly global 1979–present
- **Data variables of interest**: 
    - `msnswrf` Mean surface net short-wave radiation flux  (W/m^2)
    - `msnlwrf` Mean surface net long-wave radiation flux (W/m^2)

In [1]:
import xarray as xr
import numpy as np
import xesmf

import matplotlib.pyplot as plt

In [2]:
!ls data/*.nc

data/ERA5.nc			      data/mhw_labels.nc
data/OAFlux.nc			      data/oaflux_air_sea_fluxes_train.nc
data/era5_air_sea_fluxes_test.nc      data/preprocessed_OISST.nc
data/marine_heatwave_labels_train.nc


In [4]:
# Import data and select data between 2016–2020
ds = xr.open_dataset('data/era5.grib', engine='cfgrib').sel(time=slice('2016-01-01','2020-12-31'))

# This particular file has some variables we're not interested in. We will drop these. 
ds = ds.drop(labels=('msdwlwrf','msdwswrf'))

ds

<xarray.Dataset>
Dimensions:     (time: 60, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2016-01-31T18:00:00 ... 2020-12-31T18:0...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] ...
Data variables:
    msnlwrf     (time, latitude, longitude) float32 ...
    msnswrf     (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-09-10T17:17 GRIB to CDM+CF via cfgrib-0.9.9...

### Resample grid from 0.25º to 1º using xESMF bilinear interpolation

In [5]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.5, 1)),
                     'lon': (['lon'], np.arange(0.5, 360.5, 1)),
                    }
                   )

regridder = xesmf.Regridder(ds, ds_out, 'bilinear')
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_721x1440_180x360.nc 
Reuse pre-computed weights? False 
Input grid shape:           (721, 1440) 
Output grid shape:          (180, 360) 
Periodic in longitude?      False

In [6]:
ds_out = regridder(ds)
ds_out

/srv/conda/envs/mhw-hackathon/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


<xarray.Dataset>
Dimensions:     (time: 60, lat: 180, lon: 360)
Coordinates:
    number      int64 0
  * time        (time) datetime64[ns] 2016-01-31T18:00:00 ... 2020-12-31T18:0...
    step        timedelta64[ns] 12:00:00
    surface     float64 0.0
    valid_time  (time) datetime64[ns] 2016-02-01T06:00:00 ... 2021-01-01T06:0...
  * lon         (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat         (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    msnlwrf     (time, lat, lon) float64 -51.15 -51.17 -51.2 ... -44.1 -44.1
    msnswrf     (time, lat, lon) float64 36.88 36.88 36.88 ... 1e-15 1e-15 1e-15
Attributes:
    regrid_method:  bilinear

### Save regridded data variables to netCDF

In [7]:
era5 = xr.Dataset(
    data_vars=dict(
        msnlwrf=(['time','lat','lon'], ds_out.msnlwrf.data, {'long_name': 'Mean surface net long-wave radiation flux', 'units':'W/m^2'}),
        msnswrf=(['time','lat','lon'], ds_out.msnswrf.data, {'long_name': 'Mean surface net short-wave radiation flux', 'units':'W/m^2'})
    ),
    coords=dict(
        time=(['time'], ds_out.time.data),
        lat=(['lat'], ds_out.lat.data),
        lon=(['lon'], ds_out.lon.data),
    ),
    attrs=dict(description="European Centre for Medium-Range Weather Forecasts: ERA5"),
)

era5.to_netcdf('data/era5_air_sea_fluxes_test.nc')

era5

<xarray.Dataset>
Dimensions:  (time: 60, lat: 180, lon: 360)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-31T18:00:00 ... 2020-12-31T18:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Data variables:
    msnlwrf  (time, lat, lon) float64 -51.15 -51.17 -51.2 ... -44.11 -44.1 -44.1
    msnswrf  (time, lat, lon) float64 36.88 36.88 36.88 ... 1e-15 1e-15 1e-15
Attributes:
    description:  European Centre for Medium-Range Weather Forecasts: ERA5